In [1]:
__author__ = "Victor Xu"
__email__ = "victor.c.xu@gmail.com"
__website__ = "victorxu.me"

__copyright__ = "Copyright 2019, Victor Xu"

## Deployment

Please run the startup script to build and run the docker container. The API is exposed on port 80 and is reachable via **POST** requests.

#### Building & running the Docker container

In Terminal, type `source build_and_run.sh`

## Calling the API

In [2]:
import requests
import pandas as pd

from src.player_stat import get_player_stat

#### Players we're testing

| Name | NBA ID   | Position |Note|
|------|------|------|------|
|Lebron James|2544|Small Forward|Play style not bound by position label|
|Pascal Siakam|1627783|Power Forward|Standard power foward|
|DeMarcus Cousins|202326|Center|Standard center|
|Damian Lillard|203081|Point Guard|Standard point guard|
|Anfernee Simons|1629014|Shooting Guard|Rookie with little data|

#### Grabbing the data from NBA api and pipe through ETL

In [3]:
def test_api():
    """This function is a variation of the flask unit test.
    
    It calls the API and compares the predicted and ground truth label
    """
    
    
    player_ids_to_test = ['2544','1627783','202326','203081','1629014']
    
    player_names = ['Lebron James', 
                    'Pascal Siakam', 
                    'DeMarcus Cousins', 
                    'Damian Lillard', 
                    'Anfernee Simons']
    
    label_encoder_dict = dict([
        ("C",0),
        ("PG",1),
        ("SG",2),
        ("SF",3),
        ("PF",4)])
    
    player_labels = ['SF', 'PF', 'C', 'PG', 'SG']
    player_labels_int = [label_encoder_dict[label] for label in player_labels]

    
    # Making the API call
    header = {'Content-Type': 'application/json', 
              'Accept': 'application/json'}

    # IP of the docker container running the predict service
    request_url = "http://127.0.0.1:80/predict"
    
    results = []

    for name, pid, label in zip(player_names, player_ids_to_test, player_labels_int):
        
        player_test_data = get_player_stat(pid).to_json(orient='records')
        
        # Make the request to our dockerized model
        r = requests.post(request_url, data=player_test_data, headers=header)
        
        print("Response in JSON:", r.content)
        
        predicted_label = pd.read_json(r.content).values[0][0]
        
        results.append([name, label, predicted_label])
        
    # Construct the result dataframe
    result_df = pd.DataFrame(results, columns=['Name','Ground Truth', 'Predicted'])  
    
    label_encoder_dict_rev = dict([
        (0, "C"),
        (1, "PG"),
        (2, "SG"),
        (3, "SF"),
        (4, "PF")])
    
    for col in ['Ground Truth', 'Predicted']:
        result_df[col] = result_df[col].map(label_encoder_dict_rev)
        
    return result_df 

In [4]:
test_api()

Response in JSON: b'[{"y_pred":1}]'
Response in JSON: b'[{"y_pred":4}]'
Response in JSON: b'[{"y_pred":0}]'
Response in JSON: b'[{"y_pred":1}]'
Response in JSON: b'[{"y_pred":2}]'


,Name,Ground Truth,Predicted
0,Lebron James,SF,PG
1,Pascal Siakam,PF,PF
2,DeMarcus Cousins,C,C
3,Damian Lillard,PG,PG
4,Anfernee Simons,SG,SG
